# Export YOLO-NAS to ONNX

This tutorial demonstrates how to export a trained YOLO-NAS checkpoint to ONNX format for deployment.

In [ ]:
import torch
from pathlib import Path
from modern_yolonas import yolo_nas_s
from modern_yolonas.training import extract_model_state_dict

# Load model from checkpoint
# Replace with your checkpoint path, or use pretrained=True for COCO weights
model = yolo_nas_s(pretrained=True)
# sd = extract_model_state_dict("runs/hardhat/last.ckpt")
# model.load_state_dict(sd)
model.eval()

# Fuse RepVGG blocks for deployment (merges BN into conv)
for module in model.modules():
    if hasattr(module, "fuse_block_residual_branches"):
        module.fuse_block_residual_branches()

dummy = torch.randn(1, 3, 640, 640)
onnx_path = "model_float32.onnx"

torch.onnx.export(
    model, dummy, onnx_path,
    input_names=["images"],
    output_names=["pred_bboxes", "pred_scores"],
    dynamic_axes={
        "images": {0: "batch"},
        "pred_bboxes": {0: "batch"},
        "pred_scores": {0: "batch"},
    },
    opset_version=17,
)

onnx_size = Path(onnx_path).stat().st_size / (1024 * 1024)
print(f"ONNX exported to: {onnx_path}")
print(f"Float32 model size: {onnx_size:.1f} MB")

In [ ]:
import onnxruntime as ort

session = ort.InferenceSession(onnx_path)
outputs = session.run(None, {"images": dummy.numpy()})
print(f"Output shapes: bboxes={outputs[0].shape}, scores={outputs[1].shape}")
print("ONNX model verified successfully!")